In [1]:
import pandas as pd
import numpy as np

In [2]:
data = np.loadtxt('./data/wifi_localization.txt',dtype=int)
col = ['wifi1','wifi2','wifi3','wifi4','wifi5','wifi6','wifi7','room']
df = pd.DataFrame(data,columns=col)
df.head()

,wifi1,wifi2,wifi3,wifi4,wifi5,wifi6,wifi7,room
0,-64,-56,-61,-66,-71,-82,-81,1
1,-68,-57,-61,-65,-71,-85,-85,1
2,-63,-60,-60,-67,-76,-85,-84,1
3,-61,-60,-68,-62,-77,-90,-80,1
4,-63,-65,-60,-63,-77,-81,-87,1


In [2]:
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
X = np.random.rand(100, 1) * 10 # Random data
y = 2.5 * X.flatten() + np.random.randn(100) * 2 # Linear relationship with noise
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
random_state=42)

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")

with mlflow.start_run():
 # Define and train the model
 model = LinearRegression()
 model.fit(X_train, y_train)
 # Log parameters
 mlflow.log_param("fit_intercept", model.fit_intercept)
 
 # Predict and log metrics
 y_pred = model.predict(X_test)
 rmse = mean_squared_error(y_test, y_pred, squared=False)
 mlflow.log_metric("rmse", rmse)
 
 # Log the model
 mlflow.sklearn.log_model(model, "linear_model")



c:\Users\hiraa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/12 23:11:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run wistful-hog-928 at: http://localhost:5000/#/experiments/0/runs/4d4deb3232cf41259cb36f36a04c4799
🧪 View experiment at: http://localhost:5000/#/experiments/0


In [6]:
with mlflow.start_run():
 model = LinearRegression(fit_intercept=False)
 model.fit(X_train, y_train)
 y_pred = model.predict(X_test)
 rmse = mean_squared_error(y_test, y_pred, squared=False)
 
 mlflow.log_param("fit_intercept", model.fit_intercept)
 mlflow.log_metric("rmse", rmse)
 mlflow.sklearn.log_model(model, "linear_model_no_intercept")

c:\Users\hiraa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/12 23:14:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run respected-shrimp-869 at: http://localhost:5000/#/experiments/0/runs/3ee82bdea7ff4742895951f4323a8a38
🧪 View experiment at: http://localhost:5000/#/experiments/0
